In [7]:
import numpy as np
import pandas as pd
import transformers
import torchvision
import torch

from transformers import AutoConfig, TrainingArguments
from transformers import AutoTokenizer,AutoModelForTokenClassification
from datasets import load_dataset
from transformers import RobertaTokenizerFast
from scipy.stats import pearsonr

In [5]:
df = pd.read_csv('/Users/ginger./Desktop/Ginger/NYU/2024 Fall/Cognitive/Project/topic 2/词性数据/couplet_0_10000.csv')
df_test = pd.read_csv('/Users/ginger./Desktop/Ginger/NYU/2024 Fall/Cognitive/Project/topic 2/词性数据/couplet_340001_344949.csv')

display(df.head(5),df_test.head(5))

,Column1,Column2,Column1_Tag,Column2_Tag
0,晚风摇树树还挺,晨露润花花更红,"[0, 0, 1, 0, 0, 0, 1]","[0, 0, 1, 0, 0, 0, 0]"
1,愿景天成无墨迹,万方乐奏有于阗,"[1, 0, 0, 1, 1, 0, 0]","[0, 0, 0, 1, 1, 0, 0]"
2,丹枫江冷人初去,绿柳堤新燕复来,"[0, 0, 0, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 1]"
3,闲来野钓人稀处,兴起高歌酒醉中,"[0, 1, 0, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0]"
4,投石向天跟命斗,闭门问卷与时争,"[1, 0, 0, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 1]"


,Column1,Column2,Column1_Tag,Column2_Tag
0,半百心愁温旧梦,一江春水向东流,"[0, 0, 0, 1, 1, 0, 0]","[0, 0, 0, 0, 0, 0, 1]"
1,淤泥未染荷君子,风雨独高松大夫,"[0, 0, 0, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0]"
2,新绿水生春释菜,小红桥近夜看花,"[0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 1, 0]"
3,惠山西指九峰麓,吴会南临一宿程,"[0, 0, 0, 1, 0, 0, 0]","[0, 0, 0, 1, 0, 0, 0]"
4,匠笔濡心开绝境,师言得体亮高风,"[0, 0, 1, 0, 1, 0, 0]","[0, 0, 0, 0, 0, 0, 0]"


In [6]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("KoichiYasuoka/chinese-roberta-base-upos")
model = AutoModelForTokenClassification.from_pretrained("KoichiYasuoka/chinese-roberta-base-upos")
# Map pos information into numbers
pos_mapping = {
    'ADJ': 0, "ADP": 1, "ADV": 2,
    "AUX": 3, "CCONJ": 4, "DET": 5,
    "INTJ": 6, "NOUN": 7, "NUM": 8,
    "PART":9, "PRON": 10, "PROPN": 11,
    "PUNCT": 12, "SCONJ": 13, "SYM":14,
    "VERB": 15, "X": 16
}

config.json:   0%|          | 0.00/2.58k [00:00<?, ?B/s]

In [8]:
def pos_tagging(sentence):
    # Tokenize the input sentence (raw string, not pretokenized)
    tokens = tokenizer(sentence, return_tensors="pt", truncation=True)
    input_ids = tokens["input_ids"]
    attention_mask = tokens["attention_mask"]

    # Pass the tokens through the model
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    # Get predictions
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

    # Map tokens to labels
    
    predicted_labels = [model.config.id2label[p.item()] for p in predictions[0]]
    tokenized_words = tokenizer.convert_ids_to_tokens(input_ids[0])

    tags = list(zip(tokenized_words, predicted_labels))[1:-2]
    results = []
    for i in tags:
        tag = i[1]
        if '-' in tag:
            results.append(tag[2:])
        else:
            results.append(tag)
    return (sentence,results)

def coefficients(input1,input2):
    tags1 = input1[1]
    tags2 = input2[1]
    numerical_tags1 = [pos_mapping[tag] for tag in tags1]
    numerical_tags2 = [pos_mapping[tag] for tag in tags2]
    correlation, _ = pearsonr(numerical_tags1, numerical_tags2)
    euclidean_distance = np.linalg.norm(np.subtract(numerical_tags1, numerical_tags2))
    cosine_similarity = np.dot(numerical_tags1, numerical_tags2) / (
        np.linalg.norm(numerical_tags1) * np.linalg.norm(numerical_tags2)
)

    return (input1[0],input2[0],correlation,euclidean_distance,cosine_similarity)
    


In [13]:
a = pos_tagging(df['Column1'][0])
b = pos_tagging(df['Column2'][0])
coup1, coup2, pc,ed,cs = coefficients(a,b)
print(f"The example couplet is:\n{coup1},{coup2}")
print(f"The Pearson correlation coefficient is:\n{np.round(pc,4)}")
print(f"The Euclidean distance is:\n{np.round(ed,4)}")
print(f"The Cosine Similarity is:\n{np.round(cs,4)}")


The example couplet is:
晚风摇树树还挺,晨露润花花更红
The Pearson correlation coefficient is:
0.7157
The Euclidean distance is:
7.2111
The Cosine Similarity is:
0.9421


**TEST**

In [124]:
df_test1 = pd.read_csv('/Users/ginger./Desktop/Ginger/NYU/2024 Fall/Cognitive/Project/topic 2/词性数据/couplet_330001_340000.csv')
df_test2 = pd.read_csv('/Users/ginger./Desktop/Ginger/NYU/2024 Fall/Cognitive/Project/topic 2/词性数据/couplet_340001_344949.csv')
# df_test1.head()

In [14]:
columns = ['Column1','Column2','Pearson_coefficient','Euclidean_distance','Cosine_similarity']
results = []
for i in range(len(df_test1)):
    a = pos_tagging(df_test1['Column1'][i])
    b = pos_tagging(df_test1['Column2'][i])
    results.append(coefficients(a,b))
df_output1 = pd.DataFrame(results, columns=columns)
    

In [162]:
results = []
for i in range(len(df_test2)):
    a = pos_tagging(df_test2['Column1'][i])
    b = pos_tagging(df_test2['Column2'][i])
    results.append(coefficients(a,b))
df_output2 = pd.DataFrame(results, columns=columns)
 

/var/folders/dx/fzfq8m112llbtp804xlyjn_h0000gn/T/ipykernel_30526/3164468200.py:37: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlation, _ = pearsonr(numerical_tags1, numerical_tags2)
/var/folders/dx/fzfq8m112llbtp804xlyjn_h0000gn/T/ipykernel_30526/3164468200.py:39: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity = np.dot(numerical_tags1, numerical_tags2) / (


In [ ]:
df_output1.to_csv('couplet_pos_results1.csv', index = False)
df_output2.to_csv('couplet_pos_results2.csv', index = False)

**Calculate scores for couplet used in the survey**

In [130]:
df_survey = pd.read_csv('filtered_survey_results.csv')

In [136]:
results = []
survey_couplet = list(df_survey.columns[3:])
for i in range(len(survey_couplet)):
    a = pos_tagging(survey_couplet[i][0:7])
    b = pos_tagging(survey_couplet[i][8:])
    results.append(coefficients(a,b))
df_pos_survey = pd.DataFrame(results, columns=columns)
 

In [138]:
df_pos_survey.to_csv('pos_survey_ouput.csv')